In [2]:
import yfinance as yf
import requests
import pandas as pd
from datetime import datetime

def download_stock_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

def preprocess_data_for_csv(data, symbol, period):
    data['Symbol'] = symbol
    data['Period'] = period
    data.reset_index(inplace=True)
    data.rename(columns={'Date': 'date'}, inplace=True)
    return data

def fetch_and_process_financial_data(symbol, period, api_key):
    base_url = "https://www.alphavantage.co/query"
    functions = ["BALANCE_SHEET", "INCOME_STATEMENT", "CASH_FLOW"]

    financial_data = {'Symbol': [], 'Period': [], 'date': []}

    for function in functions:
        params = {
            "function": function,
            "symbol": symbol,
            "apikey": api_key,
        }

        response = requests.get(base_url, params=params)
        data = response.json().get(function.lower(), {})

        for date, values in data.items():
            financial_data['date'].append(date)
            financial_data['Symbol'].append(symbol)
            financial_data['Period'].append(period)
            financial_data.update({f"{function}_{key}": [values[key] for key in values]})

    return pd.DataFrame(financial_data)

def save_to_csv(data, filename):
    data.to_csv(filename, index=False)

# Example usage:
symbols = ['AAPL', 'RELIANCE.NS']
start_date = '2023-01-01'
end_date = '2023-11-30'
api_key = 'AVOBAO2X1JSBSE5K'
filename = 'financial_data_combined.csv'

# Initialize an empty dataframe to store the data
combined_data = pd.DataFrame()

for symbol in symbols:
    # Download historical stock data
    stock_data = download_stock_data(symbol, start_date, end_date)

    # Preprocess stock data for CSV
    stock_data_processed = preprocess_data_for_csv(stock_data, symbol, 'Q1/2004')

    # Fetch and process financial data
    financial_data = fetch_and_process_financial_data(symbol, 'Q1/2004', api_key)

    # Merge stock and financial data
    merged_data = pd.merge(stock_data_processed, financial_data, how='left', on='date')

    # Concatenate the data for all symbols
    combined_data = pd.concat([combined_data, merged_data], ignore_index=True)

# Save the combined data to CSV
save_to_csv(combined_data, filename)
combined_data


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,date,Open,High,Low,Close,Adj Close,Volume,Symbol_x,Period_x,Symbol_y,Period_y
0,2023-01-03,130.279999,130.899994,124.169998,125.070000,124.374802,112117500,AAPL,Q1/2004,NaN,NaN
1,2023-01-04,126.889999,128.660004,125.080002,126.360001,125.657639,89113600,AAPL,Q1/2004,NaN,NaN
2,2023-01-05,127.129997,127.769997,124.760002,125.019997,124.325081,80962700,AAPL,Q1/2004,NaN,NaN
3,2023-01-06,126.010002,130.289993,124.889999,129.619995,128.899521,87754700,AAPL,Q1/2004,NaN,NaN
4,2023-01-09,130.470001,133.410004,129.889999,130.149994,129.426559,70790800,AAPL,Q1/2004,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
443,2023-11-20,2348.550049,2358.399902,2336.399902,2349.350098,2349.350098,2245093,RELIANCE.NS,Q1/2004,NaN,NaN
444,2023-11-21,2366.000000,2388.000000,2360.199951,2378.899902,2378.899902,4107225,RELIANCE.NS,Q1/2004,NaN,NaN
445,2023-11-22,2375.000000,2394.449951,2372.199951,2388.199951,2388.199951,4267407,RELIANCE.NS,Q1/2004,NaN,NaN
446,2023-11-23,2388.199951,2400.000000,2388.199951,2395.500000,2395.500000,4265771,RELIANCE.NS,Q1/2004,NaN,NaN


In [3]:
import pandas as pd
import yfinance as yf

# Define the column names
column_names = [
    "Symbol",
    "Period",
    "Date",
    "Total Assets",
    "Total Liabilities",
    "Paid-up Capital",
    "Equity",
    "Revenue",
    "Expenses",
    "EBITDA",
    "EBIT",
    "Net Profit",
    "EPS",
    "Operating Cash Flow",
    "Investing Cash Flow",
    "Financing Cash Flow",
    "Net Cash Flow",
    "ROE",
    "ROA",
    "Gross Profit Margin",
    "Net Profit Margin",
    "EBIT Margin"
]

# Define the stocks to retrieve data for
stocks = ["AAPL", "RELIANCE.NS", "TSLA"]

# Retrieve data for each stock
data = []
for stock in stocks:
    ticker = yf.Ticker(stock)

    # Get financial statements for the most recent quarter
    financials = ticker.quarterly_financials()

    # Extract relevant data from financials
    for period, row in financials.iterrows():
        date = row["asOfDate"]
        total_assets = row["totalAssets"]
        total_liabilities = row["totalLiabilities"]
        paid_up_capital = row["paidUpCapital"]
        equity = row["equity"]
        revenue = row["revenue"]
        expenses = row["costOfRevenue"]
        ebitda = row["ebitda"]
        ebit = row["ebit"]
        net_profit = row["netIncome"]
        eps = row["earningsPerShare"]
        operating_cash_flow = row["cashFlowFromOperatingActivities"]
        investing_cash_flow = row["cashFlowFromInvestingActivities"]
        financing_cash_flow = row["cashFlowFromFinancingActivities"]
        net_cash_flow = row["netCashFlowFromOperatingActivities"]
        roe = row["roe"]
        roa = row["roa"]
        gross_profit_margin = (revenue - expenses) / revenue * 100
        net_profit_margin = net_profit / revenue * 100
        ebit_margin = ebit / revenue * 100

        # Append data to list
        data.append([
            stock,
            period,
            date,
            total_assets,
            total_liabilities,
            paid_up_capital,
            equity,
            revenue,
            expenses,
            ebitda,
            ebit,
            net_profit,
            eps,
            operating_cash_flow,
            investing_cash_flow,
            financing_cash_flow,
            net_cash_flow,
            roe,
            roa,
            gross_profit_margin,
            net_profit_margin,
            ebit_margin
        ])

# Convert data to DataFrame
df = pd.DataFrame(data, columns=column_names)

# Save DataFrame to CSV file
df.to_csv("stock_data.csv", index=False)

TypeError: 'DataFrame' object is not callable

In [7]:
import yfinance as yf

# Replace 'AAPL' with your desired stock symbol
symbol = 'AAPL'

# Create a Ticker object for the specified stock symbol
ticker = yf.Ticker(symbol)

# Get quarterly financial statements
financials = ticker.quarterly_financials

# Display the financial statements
financials


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [5]:
financials

NameError: name 'financials' is not defined